In [1555]:
!pip install alive_progress
import pandas as pd
import math
import io
from datetime import datetime, time
from alive_progress import alive_bar
import os


# Transcript Mapping

### Set File Names and Variables

Set file names here, the upload boxes will appear under each section of the Read Data section. Please make sure that the real filenames match the file names entered here. 

In [1556]:
# Controls if a sample will be used
# Set to [RANDOM, HEAD, TAIL, NONE]
FILTER = "RANDOM"
SIZE = 1000

# Training Records from SumTotal
filename = "2020-2022 Training Records.xlsx"

# Coursename Mapping Setup
USE_COURSE_MAP = True
CMAP_ORIG = "OLD_CNAME"
CMAP_NEW = "NEW_CNAME"
MAP_PATH = "map.csv"

# Saba Courses
COURSE_PATH = "saba_courses.csv"

# Saba People
PEOPLE_PATH = "saba_people.csv"

### Other Global Variables

In [1557]:
USERNAME = "Employee Number"
COURSEID = "OFFERING"
COURSEVERSION = "COURSE_VERSION"
COMPDATE = "Completion Date"
STATUS = "Completion Status"
COURSETITLE = "Activity Name"
PASSFAIL = "Pass/Fail"
ATTSTATUS = "Attendance Status"

FOLDER = "trans_output/"
OUTPUT_PRE = "transcript_out_"
ERRORS_PRE = "transcript_errors_"
MISSING_COURSE = "missing_course_"
MISSING_PEOPLE = "missing_people_"
FILE_EXT = ".csv"
TIME_SUFF = datetime.now().strftime('%Y_%m_%d-%H-%M')

## Read Data

#### Transcript List

In [1558]:
### Note: if using Google Colab, this must be set to the filename of the 
# file you will load. This is a quirk of Colab. 
try:
  import google.colab
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  source = pd.read_excel(io.BytesIO(uploaded.get(filename)))
else:
  print("Not Colab")
  source = pd.read_excel(filename)

Not Colab


#### Mapping Data

In [1559]:
def map_course(mapping_list, orig_df):
    new_df = orig_df
    for _, map_row in mapping_list.iterrows():
        old_course = map_row[CMAP_ORIG]
        new_course = map_row[CMAP_NEW]
        #new_df[[COURSEID]] = new_df[[COURSEID]].replace(old_course, new_course)
        new_df.replace(old_course, new_course, inplace=True)
        #print(old_course, new_course)
    return new_df

In [1560]:
if USE_COURSE_MAP:
  if IN_COLAB:
    print("Colab")
    uploaded = files.upload()
    course_map = pd.read_csv(io.BytesIO((uploaded.get(MAP_PATH))))
  else:
    print("Not Colab")
    course_map = pd.read_csv(MAP_PATH)

Not Colab


#### Saba Courses

In [1561]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  courses = pd.read_csv(io.BytesIO((uploaded.get(COURSE_PATH))))
else:
  print("Not Colab")
  courses = pd.read_csv(COURSE_PATH)


Not Colab


#### Saba People

In [1562]:
if IN_COLAB:
  print("Colab")
  uploaded = files.upload()
  people = pd.read_csv(io.BytesIO((uploaded.get(PEOPLE_PATH))))
else:
  print("Not Colab")
  people = pd.read_csv(PEOPLE_PATH)

Not Colab


## Begin Processing

In [1563]:
# Generate Output Filename
def genFileName(prefix, suffix, extension):
    cwd = os.getcwd()+"/"
    return cwd+FOLDER+str(prefix)+str(suffix)+str(extension)

In [1564]:
# Take Data Sample if Testing
if FILTER == "RANDOM":
    source = source.sample(SIZE)
elif FILTER == "HEAD":
    source = source.head(SIZE)
elif FILTER == "TAIL":
    source = source.tail(SIZE)


In [1565]:
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
102401,10256,"BEECHING, David",NaN,David.Beeching@cnl.ca,R&D Mechanical Technologist,220.0,MATERIALS AND FUELS PERFORMANC,"HAQUE, Zia",Versional,LGL-1003-Online,Values and Ethics at CNL,NaN,100.0,Attended,Pass,Completed,2021-11-05,2022-11-06,NaN
44185,59155,"BREEN, Ryan",NaN,Ryan.Breen@cnl.ca,Nuclear Operator Trainee,442.0,WASTE TREATMENT CENTRE OPERATI,"DOMBROSKIE, Brad",Course,NMMS-2001-Online (REV 4.3),Fissionable Materials Accounting Refresher (4.3),4.3,87.5,Attended,Pass,Completed,2021-06-21,NaT,NaN
100823,92111,"MASSE, Alexandre",NaN,alexandre.masse@cnl.ca,Project Leader,8628.0,G-1 DECOMMISSIONING,"STEVER, Matt",Versional,OSH-1004-Online,Lock Out Tag Out Exam,NaN,NaN,Attended,Pass,Completed,2021-06-07,2024-06-07,NaN
122489,92664,"FRITZ, Chris",NaN,chris.fritz@cnl.ca,Decommissioning Specialist,8571.0,NPD ENGINEERING,"PHILLIPS, Brad",Course,SECU-2008-Online (Rev 1.6),Security Awareness Refresher (1.6),1.6,NaN,Attended,NaN,Completed,2022-02-21,NaT,NaN
11626,52958,"MCFADDEN, Melanie",NaN,Melanie.McFadden@cnl.ca,Job Supervisor,8672.0,CRL REACTOR DECOMM PROGRAM EXE,"LANCE, Kacie",Course,FIRE-1001-Online (REV 4.5),Fire Prevention (4.5),4.5,90.0,Attended,Pass,Completed,2022-01-04,NaT,NaN


### Generate Status Table (Optional)

In [1566]:
#print(source.shape[1])
#source[["Completion Status", "Attendance Status", "Pass/Fail"]].groupby(["Pass/Fail","Completion Status", "Attendance Status"], dropna=False, as_index=False).size()

### Update Mapped Course Names

In [1567]:
if USE_COURSE_MAP:
    source = map_course(course_map, source)
source.head(5)

,Employee Number,Employee Name,Waived Notes,Employee email,Job Title,Department Code,Department Name,MANAGER,Activity Type,OFFERING,Activity Name,COURSE_VERSION,Score,Attendance Status,Pass/Fail,Completion Status,Completion Date,Expiration Date,Note
102401,10256,"BEECHING, David",NaN,David.Beeching@cnl.ca,R&D Mechanical Technologist,220.0,MATERIALS AND FUELS PERFORMANC,"HAQUE, Zia",Versional,LGL-1003-Online,Values and Ethics at CNL,NaN,100.0,Attended,Pass,Completed,2021-11-05,2022-11-06,NaN
44185,59155,"BREEN, Ryan",NaN,Ryan.Breen@cnl.ca,Nuclear Operator Trainee,442.0,WASTE TREATMENT CENTRE OPERATI,"DOMBROSKIE, Brad",Course,NMMS-2001-Online (REV 4.3),Fissionable Materials Accounting Refresher (4.3),4.3,87.5,Attended,Pass,Completed,2021-06-21,NaT,NaN
100823,92111,"MASSE, Alexandre",NaN,alexandre.masse@cnl.ca,Project Leader,8628.0,G-1 DECOMMISSIONING,"STEVER, Matt",Versional,OSH-1004-Online,Lock Out Tag Out Exam,NaN,NaN,Attended,Pass,Completed,2021-06-07,2024-06-07,NaN
122489,92664,"FRITZ, Chris",NaN,chris.fritz@cnl.ca,Decommissioning Specialist,8571.0,NPD ENGINEERING,"PHILLIPS, Brad",Course,SECU-2008-Online (Rev 1.6),Security Awareness Refresher (1.6),1.6,NaN,Attended,NaN,Completed,2022-02-21,NaT,NaN
11626,52958,"MCFADDEN, Melanie",NaN,Melanie.McFadden@cnl.ca,Job Supervisor,8672.0,CRL REACTOR DECOMM PROGRAM EXE,"LANCE, Kacie",Course,FIRE-1001-Online (REV 4.5),Fire Prevention (4.5),4.5,90.0,Attended,Pass,Completed,2022-01-04,NaT,NaN


### Read SABA Data

#### Course Data

In [1568]:
# Process Course Data from Saba
course_id = courses[["Course ID", "Version"]]
ids_only = pd.Series(courses["Course ID"].unique())

courses.head()

,Title,Version,Course ID,Unnamed: 3
0,100035-IBM Cognos Analytics Advanced (Cost Cou...,1.0,IT-9050,NaN
1,100035-IBM Cognos Analytics Fundamentals (Cost...,1.0,IT-9049,NaN
2,10 Commandments of Training,1.0,TD-1048,NaN
3,12 GA Shotgun,1.0,SECU-3015,NaN
4,5S Methodology (Cost Course),1.0,PI-9006-ONLINE,NaN


#### Employee Data

In [1569]:
# Process People Data from Saba
people_un = set(people["Username"].astype("int64", errors="ignore").unique())
#people.head()
len(people_un)

3869

### Data Processing Functions

In [1570]:
def procStatus(pass_fail, comp_status, att_status):
    status = -1
    ## Fail
    if pass_fail == "Fail":
        status = 400
    elif (pass_fail == "Pass") & (comp_status == "Not Completed"):
        status = 400
    #elif (math.isnan(pass_fail)) & (math.isnan(comp_status)) & ((att_status == "No-show") | (att_status == "Replaced")):
    #    status = 400
    ## Pass
    elif (pass_fail == "Pass") & (comp_status != "Not Completed"):
        status = 200
    elif (pd.isnull(pass_fail)) & (comp_status == "Completed"):
        status = 200
        #print("no p/f, status is complete")
    elif ( (pd.isnull(pass_fail)) & (pd.isnull(comp_status)) & (att_status == "Attended") ):
        status = 200
        #print("no p/f, no status, attstatus is attended")
    else:
        status = 400
    return status

def checkPerson(unList, person):
    is_in = False
    if str(person) in unList:
        is_in = True
    return is_in

def checkCourse(courseList, courseID, version):
    is_in = False
    replace_version = version
    for _, row in courseList.iterrows():
        #print(courseID, version, row["Course ID"], row["Version"])
        if (courseID == row["Course ID"]):
            #print("Course Match", courseID, version, row["Course ID"], row["Version"])
            if pd.isnull(version):
                replace_version = row["Version"]
                #print("Insert Version", replace_version, version)
            #else:
                #print("Orig Version:", replace_version, version)
            is_in = True
            #print("Match:", courseID, replace_version, version)
    return is_in, replace_version

class TranscriptRow:
    def __init__(self, username, courseID, courseVersion, compDate, status, courseTitle, inCourse, inPerson):
        self.username = username
        self.courseID = courseID.upper()
        self.courseVersion = courseVersion
        self.compDate = compDate
        self.status = status
        self.courseTitle = courseTitle
        self.inCourse = inCourse
        self.inPerson = inPerson


In [1571]:
# Process Data
source_used = source[[USERNAME, COURSEID, COURSEVERSION, COMPDATE, PASSFAIL, STATUS, ATTSTATUS, COURSETITLE]]

transcripts = []

with alive_bar(source_used.shape[0], force_tty=True) as bar_read:
    for index, trans in source_used.iterrows():
        username = trans[USERNAME]
        courseID = trans[COURSEID].upper()
        course_ver = trans[COURSEVERSION]
        comp_date = trans[COMPDATE].strftime('%Y-%m-%d')
        status = procStatus(trans[PASSFAIL], trans[STATUS], trans[ATTSTATUS])
        course_title = trans[COURSETITLE]
        in_course, rep_ver = checkCourse(course_id, courseID, course_ver)
        in_pers = checkPerson(people_un, username)

        #Update Version
        course_ver = rep_ver
        
        trans_entry = TranscriptRow(username, courseID, course_ver, comp_date, status, course_title, in_course, in_pers)
        transcripts.append(trans_entry)
        #print(courseID, type(courseID), courseID in ids_only.values)
        bar_read()
print("Transcripts Processed:", len(transcripts))

|███████████████████▏                    | ▇▅▃ 477/1000 [48%] in 29s (16.2/s, eta: 32s)                                 27/1000 [3%] in 2s (13.2/s, eta: 1:14) 81/1000 [8%] in 5s (15.8/s, eta: 59s)  ▄▆█ 149/1000 [15%] in 9s (16.1/s, eta: 53s) 184/1000 [18%] in 11s (16.5/s, eta: 50s) 192/1000 [19%] in 12s (16.6/s, eta: 49s) (16.6/s, eta: 48s) ▅▃▁ 233/1000 [23%] in 14s (16.8/s, eta: 46s)  ▂▂▄ 239/1000 [24%] in 14s (16.8/s, eta: 45s)  ▆█▆ 306/1000 [31%] in 18s (16.6/s, eta: 42s)  ▆▄▂ 315/1000 [32%] in 19s (16.6/s, eta: 41s) (16.6/s, eta: 41s)  ▁▃▅ 384/1000 [38%] in 23s (16.8/s, eta: 37s) 

### Print to File Outputs

In [ ]:
# Process a Transcript Row
def write_row(df_output, trans, test=False, subset="ALL"):
    #df = df_output
    
    inCourse = trans.inCourse
    inPerson = trans.inPerson

    tmp_dict = {
        "LEARNER": trans.username,
        "COURSE": trans.courseID,
        "COURSE_VERSION": trans.courseVersion,
        "COMPLETION_DATE": trans.compDate,
        "COMPLETION_STATUS": trans.status,
        "CREATE_ADHOC_COURSE": "FALSE",
        "COURSE_TITLE": trans.courseTitle
    }
    if test == True:
        tmp_dict = {
            "LEARNER": trans.username,
            "COURSE": trans.courseID,
            "COURSE_VERSION": trans.courseVersion,
            "COMPLETION_DATE": trans.compDate,
            "COMPLETION_STATUS": trans.status,
            "CREATE_ADHOC_COURSE": "FALSE",
            "COURSE_TITLE": trans.courseTitle,
            "IN_COURSE": trans.inCourse,
            "IN_PERSON": trans.inPerson
        }
    if subset == "ALL":
        df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "CLEAN":
        if ( (inCourse == 1) & (inPerson == 1) ):
            df_output = df_output.append(tmp_dict, ignore_index=True)
    elif subset == "MISSING":
        if ( (inCourse == 0) | (inPerson == 0) ):
            tmp_dict["COURSE_GOOD"] = bool(inCourse)
            tmp_dict["PERSON_GOOD"] = bool(inPerson)
            df_output = df_output.append(tmp_dict, ignore_index=True)
    
    return df_output
    

In [ ]:
# Output Header Values
out_header = [
    "LEARNER",
    "COURSE",
    "COURSE_VERSION",
    "COMPLETION_DATE",
    "COMPLETION_STATUS",
    "CREATE_ADHOC_COURSE",
    "COURSE_TITLE"
]

### Generate Saba Import Data Table and Write to File

In [ ]:
out = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out = write_row(out, ts, False, "CLEAN")
        bar()
out["ID"] = out.index
out.sample(10).T

|████████████████████████████████████████| 1000/1000 [100%] in 2.1s (470.12/s)                                          


,60,338,225,255,126,118,514,146,204,82
LEARNER,91556,92101,60437,60903,54310,59527,60411,58021,92260,92559
COURSE,WL-317,OSH-1042-ONLINE,ENG-1001,OSH-3005,ENG-1072,EMP-1021-CRL-VIRTUAL,OSH-9070-ONLINE,OSH-9102-ONLINE,FIRE-3001,OSH-9045-ONLINE
COURSE_VERSION,1.0,12.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
COMPLETION_DATE,2021-02-03,2021-03-22,2021-08-04,2022-01-12,2021-12-13,2022-03-24,2022-03-14,2021-07-14,2021-06-28,2022-01-12
COMPLETION_STATUS,200,200,200,200,200,200,200,200,200,400
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,Return to Work Radiation Protection Training f...,WHMIS - 2015,Eng. Change Control: Engineering/Operations,WS BLOCK - Working at Heights Practical,Engineering Change Control Update –Ensure Impl...,VIRTUAL Officer in Charge and Emergency Steward,Arc Flash Safety for Canada,Crystalline Silica Awareness,Fire Extinguisher - Practical,Hand Safety
ID,60,338,225,255,126,118,514,146,204,82


In [ ]:
print(out.shape)
#out.to_csv("out_transcript.csv", index=False)
out.to_csv(genFileName(OUTPUT_PRE, TIME_SUFF, FILE_EXT), index=False)

(530, 8)


#### Write Failiures to File

In [ ]:
out_err = pd.DataFrame(columns = out_header)

with alive_bar(len(transcripts), force_tty=True) as bar:
    for ts in transcripts:
        out_err = write_row(out_err, ts, False, "MISSING")
        bar()
out_err["ID"] = out_err.index
out_err.head(10).T

|████████████████████████████████████████| 1000/1000 [100%] in 1.7s (575.57/s)                                          s, eta: 2s) 


,0,1,2,3,4,5,6,7,8,9
LEARNER,92519,92158,61291,DS::1720,61707,50583,92658,92199,61163,58547
COURSE,OSH-1042-ONLINE (REV 12.5),EMP-1037-ONLINE (REV 0),WL-306,WL-318,ERM-1001-ONLINE (REV 1.3),OSH-9050-ONLINE,HR-1005,TD-9030-ONLINE (REV 1.3),LGL-1003-ONLINE (REV 3.2),OSH-9071-ONLINE (REV 1)
COURSE_VERSION,12.5,0.0,1.0,1.0,1.3,1.0,1.0,1.3,3.2,1.0
COMPLETION_DATE,2021-12-08,2022-01-12,2021-03-04,2021-03-22,2021-05-10,2021-01-11,2022-03-17,2021-06-02,2021-12-22,2021-05-27
COMPLETION_STATUS,200,200,200,200,400,200,200,200,200,200
CREATE_ADHOC_COURSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
COURSE_TITLE,WHMIS - 2015 (12.5),Emergency Procedure Refresher (0.0),Pre Job Brief,Return to Work Radiation Protection Training f...,Cleanup Function Overview (1.3),Lead Awareness,Mandatory Harassment and Violence Prevention T...,Harassment Prevention (1.3),Values and Ethics at CNL (3.2),Cost Course - Electrical Safety Introduction f...
COURSE_GOOD,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
PERSON_GOOD,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
ID,0,1,2,3,4,5,6,7,8,9


In [ ]:
print(out_err.shape)
#out_err.to_csv("out_errors.csv", index=False)
out_err.to_csv(genFileName(ERRORS_PRE, TIME_SUFF, FILE_EXT), index=False)

(470, 10)


### Generate Lists of Missing Courses and People

Generate list of course IDs and usernames that are in the transcript data, but do not exist in Saba. We know these will fail since the requisite data does not exist in Saba. 

In [ ]:
course_need = out_err[out_err["COURSE_GOOD"] == False]
person_need = out_err[out_err["PERSON_GOOD"] == False]

#pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv("out_courses.csv")
#pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv("out_people.csv")
pd.Series(course_need["COURSE"].unique(), name="Missing Courses").to_csv(genFileName(MISSING_COURSE, TIME_SUFF, FILE_EXT))
pd.Series(person_need["LEARNER"].unique(), name="Missing Learners").to_csv(genFileName(MISSING_PEOPLE, TIME_SUFF, FILE_EXT))